### Tanks 4.09d Vertical Fixed Roof Tank Calculations 

In [1]:
import math
import pandas as pd
import numpy as np

In [2]:
class VerticalFixedRoofTank:
    
    def __init__(self, 
                 tkshellht, 
                 skliqht, 
                 tkrfslope, 
                 diameter,
                 ins, 
                 solarabs, 
                 tax, 
                 tan, 
                 atm_p_local):
        
        '''
        tax: Average Daily Maximum Ambient Temperature, user defined, in deg F.
        tan: Average Daily Minimum Ambient Temperature, user defined, in deg F.
        solarabs: Tank surface solar absorptance, user defined, depends on tank color.
        ins: Average daily total insulation, defined to be 1,491 (btu/ft^2*d) (from Table 7.1-7)
        '''
        self.tkshellht = tkshellht
        self.skliqht = skliqht
        self.tkrfslope = tkrfslope
        self.diameter = diameter
        self.ins = ins
        self.solarabs = solarabs
        
        #Values are in deg. F
        self.tax = tax
        self.tan = tan
        
        self.rad_ = (1/2.)*self.diameter
        self.hro_ = (1/3.)*self.rad_*self.tkrfslope
        
        self.hvo = self.tkshellht - self.skliqht + self.hro_
        
        #Breather vent pressure setting (p_bp: pressure setting; p_bv: vacuum setting)
        self.p_bp = 0.03
        self.p_bv = -0.03
        
        #Returns values in def. Rankine
        self.tax_r = self.tax + 459.7
        self.tan_r = self.tan + 459.7
        
        self.delta_ta_r = self.tax_r - self.tan_r
        
        self.taa = (self.tax_r + self.tan_r) * (1/2.)
        self.tb = self.taa + (0.003*self.solarabs*self.ins)
        
        #Atmospheric pressure at facility, user defined from Table 7.1-7.
        self.atm_p_local = atm_p_local
        
    def hvo(self):
        '''
        Returns the Vapor Space Outage in units of Feet as calculated from Eqn: 1-16
        '''
        return self.hvo
        
    def vv(self):
        '''
        Returns the Vapor Space Volume of the storage tank.
        '''
        return (math.pi)*(1/4.)*(self.diameter**2)*self.hvo
    
    def tla(self):
        '''
        Returns the Daily Average Liquid Surface Temperature, as caluclated from Eqn: 1-28
        '''

        return (0.4*self.taa) + (0.6*self.tb) + ((0.005*self.solarabs)*(self.ins))
    
    def tv(self):
        '''
        Returns Average Vapor Temperature, in Rankine as caluclated from Eqn: 1-33
        '''
        return (0.7*self.taa) + (0.3*self.tb) + (0.009*self.solarabs*self.ins)
    
    def deltv(self):
        '''
        Returns the Daily Average Temperature Range, in Rankine as calculated from Eqn: 1-7
        '''
        return (0.7*self.delta_ta_r) + (0.02*self.solarabs*self.ins)
    
    def tlx_r(self):
        '''
        Returns the Maximum Liquid Temperature, in Rankine as calculated from Figure 7.1-17
        '''
        return ((0.4*self.taa) + (0.6*self.tb) + ((0.005*self.solarabs)*(self.ins))) + (0.25*((0.7*self.delta_ta_r) + (0.02*self.solarabs*self.ins)))
    
    def tln_r(self):
        '''
        Returns the Minimum Liquid Temperature, in Rankine as calculated from Figure 7.1-17
        '''
        return ((0.4*self.taa) + (0.6*self.tb) + ((0.005*self.solarabs)*(self.ins))) - (0.25*((0.7*self.delta_ta_r) + (0.02*self.solarabs*self.ins)))
    
    def tlx_f(self):
        '''
        Returns the Maximum Liquid Temperature, in F as calculated from Figure 7.1-17
        Assume: 1 R − 459.67 = -458.7 F
        '''
        return (((0.4*self.taa) + (0.6*self.tb) + ((0.005*self.solarabs)*(self.ins))) + (0.25*((0.7*self.delta_ta_r) + (0.02*self.solarabs*self.ins)))) - 458.67
    
    def tln_f(self):
        '''
        Returns the Minimum Liquid Temperature, in F as calculated from Figure 7.1-17
        Assume: 1 R − 459.67 = -458.7 F
        '''
        return (((0.4*self.taa) + (0.6*self.tb) + ((0.005*self.solarabs)*(self.ins))) - (0.25*((0.7*self.delta_ta_r) + (0.02*self.solarabs*self.ins)))) - 458.67
    
    def bventpress(self):
        '''
        Returns Breather Vent Pressure, delta_pb, calculated from Eqn: 1-10
        '''
        return self.p_bp - self.p_bv

In [3]:
tank = VerticalFixedRoofTank(tkshellht=12, skliqht=8, 
                             tkrfslope=0.0625, diameter=6,
                             ins=1491, solarabs=0.25, 
                             tax=63.5, tan=37.9, atm_p_local=12.08)

In [4]:
vv = tank.vv()
vv

114.86448139687681

In [5]:
tank.tla()

512.9346999999999

In [6]:
tank.tv()

514.0902249999999

In [7]:
tank.deltv()

25.375000000000057

In [8]:
tank.tlx_r()

519.2784499999999

In [9]:
tank.tln_r()

506.5909499999999

In [10]:
tank.tlx_f()

60.60844999999989

In [11]:
tank.tln_f()

47.92094999999989

In [12]:
tank.bventpress()

0.06

In [13]:
cyclohexane_A = 6.845
cyclohexane_B = 1203.5
cyclohexane_C = 222.86

toluene_A = 7.017
toluene_B = 1377.6
toluene_C = 222.64

benzene_A = 6.906
benzene_B = 1211.0
benzene_C = 220.79

#Converts from Rankine to deg C.
tla_c = (tank.tla() - 491.7) * (5/9.)
tla_f = (tla_c * (9/5.)) + 32
tla_f

53.23469999999992

In [14]:
tla_c

11.797055555555511

In [15]:
def calVaporPressure(an_a, an_b, an_c, tla_c):
    '''
    Returns a list of the calculated vapor pressures based on the Antone coefficients. 
    Element 0 of the list of the value in PSIA and Element 1 of the list is the value in mmHg.
    tla_c: Daily Average Liquid Surface Temperature in deg C.
    '''
    return [10**(an_a - ((an_b) / (tla_c + an_c)))/ 51.715, 10**(an_a - ((an_b) / (tla_c + an_c)))]

In [16]:
phMe_vp = calVaporPressure(an_a=toluene_A, an_b=toluene_B, an_c=toluene_C, tla_c=tla_c)
phH_vp = calVaporPressure(an_a=benzene_A, an_b=benzene_B, an_c=benzene_C, tla_c=tla_c)
cyH_vp = calVaporPressure(an_a=cyclohexane_A, an_b=cyclohexane_B, an_c=cyclohexane_C, tla_c=tla_c)

In [17]:
phMe_vp

[0.267411029342799, 13.82916138246285]

In [18]:
phH_vp

[0.9676779840276715, 50.043466943991035]

In [19]:
cyH_vp

[1.0060562857068127, 52.02820081532783]

In [20]:
comp_amt = [2812.0, 258, 101]
comp_mw = [78.11, 92.14, 84.16]
comp_name = ['Benzene', 'Toluene', 'Cyclohexane']
comp_vp = [phH_vp[0], phMe_vp[0], cyH_vp[0]]

df = pd.DataFrame({'component': comp_name, 
                   'amount_lb': comp_amt, 
                   'mw_lbs_mol': comp_mw, 
                   'comp_vp': comp_vp})

df['comp_mole'] = df['amount_lb'] / df['mw_lbs_mol']
tot_moles = np.sum(df['comp_mole'].tolist())

df['comp_mole_xi'] = df['comp_mole'] / tot_moles
df['comp_partial'] = df['comp_mole_xi'] * df['comp_vp']
vp_mixture = np.sum(df['comp_partial'].tolist())

df['comp_vap_mole_frac'] = df['comp_partial'] / vp_mixture
df['comp_vapor_mw_xi'] = df['mw_lbs_mol'] * df['comp_vap_mole_frac']
vapor_mw = np.sum(df['comp_vapor_mw_xi'].tolist())

In [21]:
vapor_mw

78.59405480262626

In [22]:
vp_mixture

0.9198100479313434

In [23]:
def stockDensity(mv, pva, tv):
    '''
    Uses R = 10.731 psia*ft3 / lb-mole* deg R
    Returns stock density in units of lbs/ft3
    '''
    return ((mv*pva) / (10.731 *tv))

In [24]:
wv = stockDensity(mv=vapor_mw, pva=vp_mixture, tv=tank.tv())
wv

0.013104133345548597

In [25]:
#TODO: Calculate vapor pressure range.

def vapPressureRange(plx, pln):
    '''
    Returns the vapor pressure range, in PSIA, as calculated from Eqn: 1-10.
    '''
    return plx - pln

In [26]:
#Converts from Rankine to deg C.
tlx_c = (tank.tlx_r() - 491.7) * (5/9.)
tln_c = (tank.tln_r() - 491.7) * (5/9.)

In [27]:
tlx_c

15.321361111111067

In [28]:
tln_c

8.272749999999956

In [29]:
phMe_vp_tlx = calVaporPressure(an_a=toluene_A, an_b=toluene_B, an_c=toluene_C, tla_c=tlx_c)
phH_vp_tlx = calVaporPressure(an_a=benzene_A, an_b=benzene_B, an_c=benzene_C, tla_c=tlx_c)
cyH_vp_tlx = calVaporPressure(an_a=cyclohexane_A, an_b=cyclohexane_B, an_c=cyclohexane_C, tla_c=tlx_c)

In [30]:
phMe_vp_tln = calVaporPressure(an_a=toluene_A, an_b=toluene_B, an_c=toluene_C, tla_c=tln_c)
phH_vp_tln = calVaporPressure(an_a=benzene_A, an_b=benzene_B, an_c=benzene_C, tla_c=tln_c)
cyH_vp_tln = calVaporPressure(an_a=cyclohexane_A, an_b=cyclohexane_B, an_c=cyclohexane_C, tla_c=tln_c)

In [31]:
comp_vp_tln = [phH_vp_tln[0], phMe_vp_tln[0], cyH_vp_tln[0]]
comp_vp_tlx = [phH_vp_tlx[0], phMe_vp_tlx[0], cyH_vp_tlx[0]]

df['comp_vp_tln'] = comp_vp_tln
df['comp_vp_tlx'] = comp_vp_tlx

In [32]:
df['comp_partial_tln'] = df['comp_mole_xi'] * df['comp_vp_tln']
df['comp_partial_tlx'] = df['comp_mole_xi'] * df['comp_vp_tlx']
df

,component,amount_lb,mw_lbs_mol,comp_vp,comp_mole,comp_mole_xi,comp_partial,comp_vap_mole_frac,comp_vapor_mw_xi,comp_vp_tln,comp_vp_tlx,comp_partial_tln,comp_partial_tlx
0,Benzene,2812.0,78.11,0.967678,36.000512,0.899997,0.870907,0.946834,73.957207,0.804679,1.157305,0.724209,1.041571
1,Toluene,258.0,92.14,0.267411,2.800087,0.070001,0.018719,0.020351,1.875138,0.217517,0.326745,0.015226,0.022872
2,Cyclohexane,101.0,84.16,1.006056,1.200095,0.030002,0.030184,0.032815,2.761709,0.840270,1.198150,0.025210,0.035947


In [33]:
tot_vp_tln = np.sum(df['comp_partial_tln'].tolist())
tot_vp_tlx = np.sum(df['comp_partial_tlx'].tolist())

In [34]:
delpv = vapPressureRange(plx=tot_vp_tlx, pln=tot_vp_tln)

In [35]:
def vaporSpaceExpansionFactor(deltv, tla, delpv, delbpv, atmp, pva):
    '''
    Returns the Vapor Space Expansion Factor (ke), as calculated from Eqn: 1-5.
    '''
    return (deltv/tla) + ((delpv - delbpv) / (atmp - pva))
ke = vaporSpaceExpansionFactor(deltv=tank.deltv(), 
                          tla=tank.tla(), 
                          delpv=delpv, 
                          delbpv=tank.bventpress(), 
                          atmp=12.08, 
                          pva=vp_mixture)
ke

0.07417818796482606

In [36]:
def ventedVaporSpaceSatFactor(pva, hvo):
    return 1/(1 + (0.053*pva*hvo))

In [37]:
ks = ventedVaporSpaceSatFactor(pva=vp_mixture, hvo=tank.hvo)

In [38]:
def calculateStandingLosses(vv, wv, ke, ks):
    '''
    Returns the standing losses from the tank in lbs/year.
    '''
    return 365*vv*wv*ke*ks

In [39]:
calculateStandingLosses(vv=vv, wv=wv, ke=ke, ks=ks)

34.01648485499338

In [40]:
def calculateWorkingLosses(vq, kn, kp, wv, kb):
    '''
    Returns the working losses from the tank in lbs/year.
    '''
    return vq*kn*kp*wv*kb